In [9]:
import os
import openai
import re
import json
from dotenv import load_dotenv
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI
from config import WHITE, GREEN, RESET_COLOR, model_name

# Define your OpenAI API key


In [19]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Function to read ctags file and extract function names and locations
import json

def extract_functions_from_json_ctags(ctags_json_file):
    functions = []

    with open(ctags_json_file, 'r') as file:
        ctags_data = json.load(file)

    for entry in ctags_data:
        if entry.get('kind') == 'function':
            function_name = entry.get('name')
            file_path = entry.get('path')
            line_number = entry.get('line')
            functions.append((function_name, file_path, line_number))

    return functions

# Function to read call hierarchy JSON file
def read_call_hierarchy_json(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
    # Process the JSON data as needed to extract relevant information
    return data  # Return the loaded JSON data


In [11]:
# Function to generate text using OpenAI API with text splitter
def generate_text_with_splitter(prompt):
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=100  # Adjust the max_tokens as needed
    )
    return response.choices[0].text.strip()

      
# Function to add a docstring before a function
def add_docstring_to_function(file_path, function_name, docstring):
    with open(file_path, 'r') as source_file:
        lines = source_file.readlines()

    in_function = False
    modified_lines = []

    for line in lines:
        # Check if the current line contains the function name
        if re.match(r'^\s*def\s+' + re.escape(function_name) + r'\s*\(.*\):', line):
            in_function = True
            modified_lines.append(f'def {function_name}():\n')
            modified_lines.append(f'    """{docstring}"""\n')  # Add the docstring
        elif in_function:
            # Copy the rest of the function
            modified_lines.append(line)
        else:
            modified_lines.append(line)

    with open(file_path, 'w') as source_file:
        source_file.writelines(modified_lines)

In [12]:

# Main function
folder_path = "/home/phucsaiyan/Documents/stage/test_addcomment2"  # Specify the folder path here
ctags_file = os.path.join(folder_path, "/home/phucsaiyan/Documents/stage/test_addcomment2/tags.json")
json_file = os.path.join(folder_path, "/home/phucsaiyan/Documents/stage/test_addcomment2/full_call_hierarchy.json")

try:
    functions = extract_functions_from_json_ctags(ctags_file)
    call_hierarchy_data = read_call_hierarchy_json(json_file)
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith(".py"):
                code_file_path = os.path.join(root, file_name)
                for function_name, _ in functions:
                    # Use the call hierarchy data to determine which functions need docstrings
                    if function_name in call_hierarchy_data:
                        prompt = f"Generate a comment for the function {function_name} based on call hierarchy data:\n\n{call_hierarchy_data}"
                        generated_text = generate_text_with_splitter(prompt)
                        add_docstring_to_function(code_file_path, function_name, generated_text)
except Exception as e:
    print(f"An error occurred: {str(e)}")

An error occurred: Extra data: line 2 column 1 (char 93)


In [15]:
print(functions)
call_hierarchy_data

[('!_TAG_EXTRA_DESCRIPTION', '/Include tags for non-named objects like lambda/'), ('!_TAG_EXTRA_DESCRIPTION', '/Include tags of file scope/'), ('!_TAG_EXTRA_DESCRIPTION', '/Include pseudo tags/'), ('!_TAG_EXTRA_DESCRIPTION', '/Include tags generated by subparsers/'), ('!_TAG_FIELD_DESCRIPTION', '/the last modified time of the input file (only for F\\/file kind tag)/'), ('!_TAG_FIELD_DESCRIPTION', '/File-restricted scoping/'), ('!_TAG_FIELD_DESCRIPTION', '/input file/'), ('!_TAG_FIELD_DESCRIPTION', '/tag name/'), ('!_TAG_FIELD_DESCRIPTION', '/pattern/'), ('!_TAG_FIELD_DESCRIPTION', '/Type and name of a variable or typedef/'), ('!_TAG_FIELD_DESCRIPTION!Python', '/the original name for the tag/'), ('!_TAG_FILE_FORMAT', '/extended format; --format=1 will not append ;" to lines/'), ('!_TAG_FILE_SORTED', '/0=unsorted, 1=sorted, 2=foldcase/'), ('!_TAG_KIND_DESCRIPTION!Markdown', '/level 2 sections/'), ('!_TAG_KIND_DESCRIPTION!Markdown', '/level 4 sections/'), ('!_TAG_KIND_DESCRIPTION!Markdown

{'add_docstring_to_function': {'name': 'add_docstring_to_function',
  'type': 'function',
  'calls': [{'name': 'open',
    'type': 'call',
    'line_number': 33,
    'column_offset': 9,
    'children': []},
   {'name': 'open',
    'type': 'call',
    'line_number': 51,
    'column_offset': 9,
    'children': []}],
  'children': []},
 '__init__': {'name': '__init__',
  'type': 'function',
  'calls': [],
  'children': []},
 'ask_question': {'name': 'ask_question',
  'type': 'function',
  'calls': [{'name': 'search_documents',
    'type': 'call',
    'line_number': 18,
    'column_offset': 20,
    'children': []},
   {'name': 'format_documents',
    'type': 'call',
    'line_number': 20,
    'column_offset': 25,
    'children': []}],
  'children': []},
 'extract_functions_from_ctags': {'name': 'extract_functions_from_ctags',
  'type': 'function',
  'calls': [{'name': 'open',
    'type': 'call',
    'line_number': 13,
    'column_offset': 9,
    'children': []},
   {'name': 'len',
    'typ

In [20]:
functions = extract_functions_from_ctags('/home/phucsaiyan/Documents/stage/test_addcomment2/tags.json')
for function in functions:
    print(f"Function Name: {function[0]}, File: {function[1]}, Line Number: {function[2]}")

In [64]:
import os
import uuid
import subprocess
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
from langchain.document_loaders import DirectoryLoader, NotebookLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from utils import clean_and_tokenize
import json
import glob 
from tabulate import tabulate
def load_folder_documents(folder_path):
    extensions = ['txt', 'md', 'markdown', 'rst', 'py', 'js', 'java', 'c', 'cpp', 'cs', 'go', 'rb', 'php', 'scala', 'html', 'htm', 'xml', 'json', 'yaml', 'yml', 'ini', 'toml', 'cfg', 'conf', 'sh', 'bash', 'css', 'scss', 'sql', 'gitignore', 'dockerignore', 'editorconfig', 'ipynb']
    documents_dict = {}

    for ext in extensions:
        glob_pattern = f'**/*.{ext}'
        try:
            loader = None
            if ext == 'ipynb':
                loader = NotebookLoader(folder_path, include_outputs=True, max_output_length=20, remove_newline=True)
            else:
                loader = DirectoryLoader(folder_path, glob=glob_pattern, loader_cls=TextLoader)

            loaded_documents = loader.load() if callable(loader.load) else []
            
            if loaded_documents:
                for doc in loaded_documents:
                    file_path = doc.metadata['source']
                    relative_path = os.path.relpath(file_path, folder_path)
                    file_id = str(uuid.uuid4())
                    doc.metadata['source'] = relative_path
                    doc.metadata['file_id'] = file_id

                    documents_dict[file_id] = doc
        except Exception as e:
            print(f"Error loading files with pattern '{glob_pattern}': {e}")
            print(f"Problematic folder path: {folder_path}")
            continue

    return list(documents_dict.values())

def load_and_index_files(repo_path):
    main_documents = load_folder_documents(repo_path)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)

    split_documents = []
    for original_doc in main_documents:
        split_docs = text_splitter.split_documents([original_doc])
        for split_doc in split_docs:
            split_doc.metadata['file_id'] = original_doc.metadata['file_id']
            split_doc.metadata['source'] = original_doc.metadata['source']

        split_documents.extend(split_docs)

    index = None
    if split_documents:
        tokenized_documents = [clean_and_tokenize(doc.page_content) for doc in split_documents]
        index = BM25Okapi(tokenized_documents)
    
    file_sources = [doc.metadata['source'] for doc in split_documents]
    # Create a list of lists to represent your data
    data = []
    for doc in split_documents:
        data.append([doc.metadata['file_id'], doc.metadata['source'], doc.page_content])

    return index, split_documents, file_sources, [doc.metadata['source'] for doc in split_documents]


ctag = '/home/phucsaiyan/Documents/stage/test_addcomment2'

a,b,c,d = load_and_index_files(ctag)


Error loading files with pattern '**/*.ipynb': [Errno 21] Is a directory: '/home/phucsaiyan/Documents/stage/test_addcomment2'
Problematic folder path: /home/phucsaiyan/Documents/stage/test_addcomment2


In [134]:
import os
import json
from langchain.document_loaders import DirectoryLoader, NotebookLoader,TextLoader
import json

folder_path = '/home/phucsaiyan/Documents/stage/test_addcomment2'
# Assuming you have already loaded the documents using the provided code
loaded_documents = load_folder_documents(folder_path)
def load_folder_documents(folder_path):
    extensions = ['txt', 'md', 'markdown', 'rst', 'py', 'js', 'java', 'c', 'cpp', 'cs', 'go', 'rb', 'php', 'scala', 'html', 'htm', 'xml', 'json', 'yaml', 'yml', 'ini', 'toml', 'cfg', 'conf', 'sh', 'bash', 'css', 'scss', 'sql', 'gitignore', 'dockerignore', 'editorconfig', 'ipynb']
    documents_dict = {}

    for ext in extensions:
        glob_pattern = f'**/*.{ext}'
        try:
            loader = None
            if ext == 'ipynb':
                loader = NotebookLoader(folder_path, include_outputs=True, max_output_length=20, remove_newline=True)
            else:
                loader = DirectoryLoader(folder_path, glob=glob_pattern, loader_cls=TextLoader)

            loaded_documents = loader.load() if callable(loader.load) else []
            
            if loaded_documents:
                for doc in loaded_documents:
                    file_path = doc.metadata['source']
                    relative_path = os.path.relpath(file_path, folder_path)
                    file_id = str(uuid.uuid4())
                    doc.metadata['source'] = relative_path
                    doc.metadata['file_id'] = file_id

                    documents_dict[file_id] = doc
        except Exception as e:
            print(f"Error loading files with pattern '{glob_pattern}': {e}")
            print(f"Problematic folder path: {folder_path}")
            continue

    return list(documents_dict.values())

Error loading files with pattern '**/*.ipynb': [Errno 21] Is a directory: '/home/phucsaiyan/Documents/stage/test_addcomment2'
Problematic folder path: /home/phucsaiyan/Documents/stage/test_addcomment2


In [135]:
loaded_documents

[Document(page_content='from main import main\n\nmain()', metadata={'source': 'app.py', 'file_id': 'a96b5ed6-5d84-4781-8260-1fd61efddbd8'}),
 Document(page_content='# file_processing.py\nimport os\nimport uuid\nimport subprocess\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.metrics.pairwise import cosine_similarity\nfrom rank_bm25 import BM25Okapi\nfrom langchain.document_loaders import DirectoryLoader, NotebookLoader\nfrom langchain.text_splitter import RecursiveCharacterTextSplitter\nfrom utils import clean_and_tokenize\n\ndef clone_github_repo():\n    """This function appears to clone a github repository."""\n    """The clone_github_repo function is used to clone a GitHub repository. This function is called by the main function."""\n    """This function is responsible for cloning a GitHub repository."""\n    try:\n        subprocess.run([\'git\', \'clone\', github_url, local_path], check=True)\n        return True\n    except subprocess.CalledProcessErro

In [140]:
import json
def search_function_names(json_data):
    function_names = []

    def search(json_obj):
        if isinstance(json_obj, dict):
            for key, value in json_obj.items():
                if callable(value) and key not in function_names:
                    function_names.append(key)
                search(value)
        elif isinstance(json_obj, list):
            for item in json_obj:
                search(item)

    search(json_data)
    return function_names
search_function_names(loaded_documents)

[]

In [138]:
import json

def search_function_names_in_ctagfile(ctagfile_path):
    # Read the JSON data from the ctagfile
    with open(ctagfile_path, 'r') as file:
        ctag_data = json.load(file)

    function_names = []

    # Assuming the JSON structure, iterate through the keys and values
    for key, value in ctag_data.items():
        if isinstance(value, dict):
            # Check if the key represents a function name
            if 'kind' in value and value['kind'] == 'function':
                function_names.append(key)

    return function_names

# Example usage
ctagfile_path = '/home/phucsaiyan/Documents/stage/test_addcomment2/output.json'
function_names = search_function_names_in_ctagfile(ctagfile_path)
print(function_names)


AttributeError: 'list' object has no attribute 'items'

In [151]:
import json

# Specify the path to your JSONL file
jsonl_file_path = '/home/phucsaiyan/Documents/stage/test_addcomment2/tags.jsonl'

# Initialize a list to store the JSON objects
json_data_list = []

# Open and read the JSONL file line by line
with open(jsonl_file_path, 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append it to the list
        json_data = json.loads(line)
        json_data_list.append(json_data)

# Now you have a list of JSON objects in json_data_list
# You can access and manipulate the data as needed

In [148]:
type(json_data_list)

list

In [153]:
import json

# Open JSONL file for reading
with open('/home/phucsaiyan/Documents/stage/test_addcomment2/tags.jsonl', 'r') as jsonl_file:
    # Read and convert JSONL data to a list of dictionaries
    json_data = [json.loads(line) for line in jsonl_file]

# Write the converted data to a standard JSON file
with open('output.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=4)
    
       
            

Full path: /home/phucsaiyan/Documents/stage/test_addcomment2/file.json


str

In [1]:
import json
with open('/home/phucsaiyan/Documents/stage/test_addcomment2/output.json', 'r') as json_file:
    data = json.load(json_file)

function_names = []
for item in data:
    if item['_type'] == 'tag' and item['kind'] == 'function':
        function_names.append(item['name'])
for function_name in function_names:
    print(function_name)


clone_github_repo
load_and_index_files
load_folder_documents
search_documents


In [3]:
def extract_function_names_from_json(json_file_path):
    # Open the JSON file and load its data
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

    # Initialize a list to store function names
    function_names = []

    # Iterate through the data and extract function names
    for item in data:
        if item.get('_type') == 'tag' and item.get('kind') == 'function':
            function_names.append(item.get('name'))

    return function_names

In [5]:
json_file_path= '/home/phucsaiyan/Documents/stage/clonechatgpt/ctag.json'
a= extract_function_names_from_json(json_file_path)
len(a)

21